<a href="https://colab.research.google.com/github/Quikii/PhD-Thesis-EU-Solidarity-Statements/blob/main/EUSpeech_Preliminary_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Import Data

0.1 EUSpeech Sample

In [ ]:
from pandas import read_csv
df = read_csv("/content/EUSpeech_translated.csv")


0.2 IOParlspeech

In [ ]:
!pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
import pyreadr

# Create a list of country files
files = [
    "/content/ioparlspeech.aus.rds",
    "/content/ioparlspeech.can.rds",
    "/content/ioparlspeech.deu.rds",
    "/content/ioparlspeech.gbr.rds",
    "/content/ioparlspeech.nzl.rds",
]

# Initialize list to store dataframes
dfs = []

# Load each file
for file in files:
    print(f"Loading {file.split('/')[-1]}...")
    result = pyreadr.read_r(file)
    temp_df = result[None]
    # Add country identifier from filename
    country = file.split('ioparlspeech.')[-1].split('.')[0].upper()
    temp_df['country'] = country
    dfs.append(temp_df)
    print(f"Successfully loaded {country}")

# Combine all dataframes
import pandas as pd
df = pd.concat(dfs, ignore_index=True)
print("\nAll data loaded and combined!")
print(f"\nDataset Overview:")
print(f"Total number of rows: {len(df):,}")
print("\nRows per country:")
print(df['country'].value_counts())

Loading ioparlspeech.aus.rds...
Successfully loaded AUS
Loading ioparlspeech.can.rds...
Successfully loaded CAN
Loading ioparlspeech.deu.rds...
Successfully loaded DEU
Loading ioparlspeech.gbr.rds...
Successfully loaded GBR
Loading ioparlspeech.nzl.rds...
Successfully loaded NZL

All data loaded and combined!

Dataset Overview:
Total number of rows: 527,141

Rows per country:
country
GBR    280602
CAN    101371
DEU     76876
AUS     42450
NZL     25842
Name: count, dtype: int64


0.1 Download Libraries

1.0 Import Libraries

In [ ]:
# @title
import re
import pandas as pd
import spacy
#import coreferee
import concurrent.futures
from tqdm import tqdm  # For progress display

1.1 Load Spacy Models


1.1.1 Tokenize into Sentences

In [ ]:
import re
import pandas as pd
import nltk

# Ensure required NLTK resources are available
try:
    nltk.data.find("tokenizers/punkt_tab/english")
except LookupError:
    nltk.download("punkt_tab")

import re
from nltk.tokenize import word_tokenize, sent_tokenize


# ----------------------------
# Function to get rokenized words and sentences
# ----------------------------
def tokenize_text(df, text_col="text.en", pattern_str=r"\bsolidar\w*\b"):
    """
    Filters rows containing a specified pattern in the text, tokenizes the text into words,
    and splits the text into sentences.

    Parameters:
        df (pd.DataFrame): Input DataFrame with a text column.
        text_col (str): Name of the column containing text (default "text.en").
        pattern_str (str): Regex pattern string to filter rows (default searches for words starting with "solidar").

    Returns:
        pd.DataFrame: A copy of the DataFrame with two new columns:
            - 'tokenized': List of tokens from the text.
            - 'sentences': List of sentences from the text.
    """
    # Compile the regex pattern
    pattern = re.compile(pattern_str, re.IGNORECASE)

    # Filter rows that match the pattern in the specified text column
    df_filtered = df[df[text_col].apply(lambda x: bool(pattern.search(x)))].copy()

    # Tokenize the text into words and store in a new column 'tokenized'
    df_filtered['tokenized'] = df_filtered[text_col].apply(word_tokenize)

    # Split the text into sentences and store in a new column 'sentences'
    df_filtered['sentences'] = df_filtered[text_col].apply(sent_tokenize)

    return df_filtered


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


1.2 Resolve Coreference Function

In [ ]:
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl#egg=en_coreference_web_trf


In [ ]:
import spacy
!pip show spacy
import spacy_experimental
nlp = spacy.load("en_coreference_web_trf")


Name: spacy
Version: 3.4.4
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi
Required-by: en-coreference-web-trf, fastai, spacy-experimental, spacy-transformers


In [ ]:
def append_corefs(doc):
    """
    Returns a version of the document text where for each non-main coreferent mention,
    the main mention is appended in square brackets.

    For example, if a mention is "he" and the main mention for that coreference chain is "Fidel Castro",
    the function will change "he" to "he [Fidel Castro]".

    Args:
        doc (spacy.tokens.Doc): A spaCy document processed with a coreference model that sets doc.spans.

    Returns:
        str: The modified text with coreference annotations appended.
    """
    resolved_text = doc.text
    offset = 0
    insertions = []

    # Iterate over each coreference chain in doc.spans.
    for chain in doc.spans:
        # Process only chains that begin with the expected prefix.
        if not chain.startswith("coref_clusters"):
            continue

        # The first mention is the main reference.
        main_text = doc.spans[chain][0].text

        # For every subsequent mention, we want to append the main text in brackets.
        for idx, span in enumerate(doc.spans[chain]):
            if idx > 0:
                # Insert after the mention; use span.end_char as the insertion point.
                insertions.append([span.end_char, f" [{main_text}]"])

    # Sort the insertions by the character index.
    for pos, insertion in sorted(insertions, key=lambda x: x[0]):
        resolved_text = resolved_text[:pos + offset] + insertion + resolved_text[pos + offset:]
        offset += len(insertion)

    return resolved_text

# Example usage:
import spacy
import spacy_experimental

nlp = spacy.load("en_coreference_web_trf")
text = """Fidel Castro led a communist revolution that toppled the Cuban government in 1959, after which he declared himself prime minister. He held the title until 1976, when it was abolished and he became head of the Communist Party and president of the council of state and the council of ministers. With his health failing, Castro handed power to his brother, Raúl, in 2006. He died in 2016."""
doc = nlp(text)
resolved = append_corefs(doc)
print("Original text:")
print(doc.text)
print("\nText with coreferences appended:")
print(resolved)


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Original text:
Fidel Castro led a communist revolution that toppled the Cuban government in 1959, after which he declared himself prime minister. He held the title until 1976, when it was abolished and he became head of the Communist Party and president of the council of state and the council of ministers. With his health failing, Castro handed power to his brother, Raúl, in 2006. He died in 2016.

Text with coreferences appended:
Fidel Castro led a communist revolution that toppled the Cuban government in 1959, after which he declared [Fidel Castro led] himself prime [Fidel Castro led] minister. He held [Fidel Castro led] the title until [himself prime minister.] 1976, when it was [himself prime minister.] abolished and he became [Fidel Castro led] head of the Communist Party and president of the council of state and the council of ministers. With his health [Fidel Castro led] failing, Castro handed [Fidel Castro led] power to his brother [Fidel Castro led], Raúl, in 2006. He died [Fi

1.3 Define Processing Function

In [ ]:

# ----------------------------
# Define processing function for each text entry
# ----------------------------

# ----------------------------
# Define processing function for each text entry
# ----------------------------
def process_text(text):
    """
    Process a single text:
      - Extract Named Entities, subject, and object from the original text.
      - Resolve coreference using coreferee (via append_corefs) on the entire text.

    Returns:
        tuple: (sentence_records, resolved_text)

        - sentence_records: A list of sentence-level dictionaries with keys:
            "sentence", "entities", "subject", and "object".
        - resolved_text: The entire text with coreference annotations appended.
    """
    # Process the original text
    doc = nlp(text)

    sentence_records = []
    # Extract sentence-level data from the original text
    for sent in doc.sents:
        sent_text = sent.text.strip()
        if not sent_text:
            continue

        sent_doc = nlp(sent_text)
        entities = [(ent.text, ent.label_) for ent in sent_doc.ents]

        subject = None
        obj = None
        for token in sent_doc:
            if token.dep_ in ("nsubj", "nsubjpass") and subject is None:
                subject = token.text
            if token.dep_ in ("dobj", "pobj") and obj is None:
                obj = token.text

        sentence_records.append({
            "sentence": sent_text,
            "entities": entities,
            "subject": subject,
            "object": obj
        })

    # Apply coreference resolution using append_corefs
    resolved_text = append_corefs(doc)

    return sentence_records, resolved_text



1.4 Run the Pre-Processing

In [ ]:
import re
# ----------------------------
# Filter DataFrame rows by "solidarity" (or related forms)
# ----------------------------

pattern = re.compile(r"\bsolidar\w*\b", re.IGNORECASE)
filtered_df = df[df["text.en"].apply(lambda x: bool(pattern.search(x)))].copy()

# ----------------------------
# Parallel processing over the filtered texts with progress display
# ----------------------------

results_sentences = []
results_resolved = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_text = {executor.submit(process_text, text): text for text in filtered_df["text.en"]}
    for future in tqdm(concurrent.futures.as_completed(future_to_text),
                       total=len(future_to_text),
                       desc="Processing texts"):
        try:
            sentence_records, resolved_text = future.result()
            results_sentences.extend(sentence_records)
            results_resolved.append(resolved_text)
        except Exception as exc:
            print(f"Error processing a text: {exc}")



# ----------------------------
# Create a DataFrame of sentence-level records
# ----------------------------

df_sentences = pd.DataFrame(results_sentences)
print(df_sentences.head())

Processing texts:   0%|          | 0/2256 [00:42<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-30-7095f0cc4be9>", line 17, in <cell line: 0>
    for future in tqdm(concurrent.futures.as_completed(future_to_text),
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/lib/python3.11/concurrent/futures/_base.py", line 243, in as_completed
    waiter.event.wait(wait_timeout)
  File "/usr/lib/python3.11/threading.py", line 629, in wait
    signaled = self._cond.wait(timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/threading.py", line 327, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-7095f0cc4be9>", line 15, in <cell line: 0>
    with concurrent

TypeError: object of type 'NoneType' has no len()

2. Running LLM

2.1 2.0 Download Libraries

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 857.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

2.1 Initialize Unsloth

2.1.1 Load Model and Tokenizer

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2000 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


2.1.2 Set-up the Peft Model

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


2.1.3 Test Whether it Works

In [6]:
instruction = (
    "You are an expert researcher in political science. "
    "Analyze the following text and determine which of the following solidarity categories best describe it: "
    "Sentence to be analyzed: We as a Community of European States must hold together in times of crisis"
    "POSSIBLE LABELS:"
    "1. National Solidarity"
    "2. European-Level Solidarity,"
    "3. Outside-Europe Solidarity. "
    "Decide which label is most suitable and provide the output in a JSON format like this:"
    "{\"label\": \"<Insert one label: National Solidarity, European-Level Solidarity, or Outside-Europe Solidarity>\"}"
    "ONLY WRITE IN THE JSON FORMAT. GENERATE NOTHING ELSE."
)
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": "We as a Community of European States must hold together in times of crisis"}
]
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 300,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

{"label": "European-Level Solidarity"}<|eot_id|>


2.2 Prompt for Free-Coding

2.2.1 Write out the prompt

In [ ]:
def code_text_with_theme(input_text, model, tokenizer):
    """
    Function that instructs the model to categorize the given text into solidarity categories.

    Parameters:
        input_text (str): The text to be analyzed.
        model: The language model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The coded response from the model containing one or more solidarity category labels.
    """

    # Optimized instruction for categorizing text into solidarity categories
    instruction = (
        "You are an expert researcher in political science. "
        "Analyze the following text and determine which of the following solidarity categories best describe it: "
        "National Solidarity, European-Level Solidarity, Outside-Europe Solidarity. "
        "If the text aligns with more than one category, return a comma-separated list of the applicable categories. "
        "Return only the category labels with no additional commentary."
        "Write Neutral, if the sentence cannot be linked to solidarity."
    )

    # Structuring the message with system and user roles
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": input_text}
    ]

    # Creating the text-generation pipeline with a deterministic setting
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, temperature=0.9)
    output = pipe(messages)

    # Return the generated response containing the solidarity categories
    return output[0]['generated_text']


2.3 Prompt for Specific Codes

In [ ]:
def code_solidarity_categories(input_text, model, tokenizer):
    """
    Function that instructs the model to categorize the given text into solidarity categories.

    Parameters:
        input_text (str): The text to be analyzed.
        model: The language model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The coded response from the model containing one or more solidarity category labels.
    """

    # Optimized instruction for categorizing text into solidarity categories
    instruction = (
        "You are an expert researcher in political science. "
        "Analyze the following text and determine which of the following solidarity categories best describe it: "
        "National Solidarity, European-Level Solidarity, Outside-Europe Solidarity. "
        "If the text aligns with more than one category, return a comma-separated list of the applicable categories. "
        "Return only the category labels with no additional commentary."
        "Write Neutral, if the sentence cannot be linked to solidarity."
    )

    # Structuring the message with system and user roles
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": input_text}
    ]

    # Creating the text-generation pipeline with a deterministic setting
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, temperature=0.9)
    output = pipe(messages)

    # Return the generated response containing the solidarity categories
    return output[0]['generated_text']


2.4 Prompt for Solidarity Speech Acts

In [ ]:
def code_solidarity_speech_acts(input_text, model, tokenizer):
    """
    Function that instructs the model to analyze political text and extract solidarity-related speech acts,
    returning a structured JSON array with the identified information.

    Parameters:
        input_text (str): The political text to be analyzed.
        model: The language model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The coded response from the model as a structured JSON array.
    """

    # Optimized instruction for extracting solidarity-related speech acts with structured output
    instruction = (
        "You are an expert in political science and computational discourse analysis. "
        "Your task is to analyze the following political text to identify and code all instances of solidarity-related speech acts. "
        "A valid solidarity-related speech act must meet these criteria: "
        "1. Transitive Structure: The utterance must contain a transitive verb structure linking a subject (the provider, typically the speaker) to a direct object (the recipient). "
        "2. Positive Orientation: The subject expresses a positive orientation (they offer some form of assistance or help, even if verbal) toward the recipient, who is understood to be in a comparatively difficult situation. "
        "3. Illocutionary Type: The speech act must clearly fall into one of the following categories: "
        "   - Solidarity Expressives: Utterances that communicate emotional support or compassion without calling for specific action. "
        "   - Solidarity Directives: Utterances that demand, request, or call on others to act in support of the recipient. "
        "   - Solidarity Commissives: Utterances in which the speaker commits themselves to take future actions to support the recipient. "
        "For each solidarity-related speech act identified in the text, extract and return the following information: "
        "   - SpeechActType: Label the act as 'Expressive,' 'Directive,' or 'Commissive.' "
        "   - Provider: Identify the actor (usually the speaker) performing the solidarity action. "
        "   - Recipient: Identify the actor (direct object) who is in need and toward whom the positive orientation is directed. "
        "   - PropositionalContent: Summarize the core message or demand embedded in the speech act. "
        "If a sentence does not contain a solidarity-related speech act or lacks the required transitive structure, do not return anything. "
        "Return your analysis as a structured JSON array where each element represents an identified speech act with the keys "
        "'SpeechActType,' 'Provider,' 'Recipient,' and 'PropositionalContent.' "
        "Do not include any additional commentary or text beyond this structured output. "
        "For example, your output should look like this:\n"
        "[\n"
        "  {\n"
        "    \"SpeechActType\": \"\",\n"
        "    \"Provider\": \"\",\n"
        "    \"Recipient\": \"\",\n"
        "    \"PropositionalContent\": \"\"\n"
        "  }\n"
        "]\n"
        "Text to analyze:\n"
        f"{input_text}"
    )

    # Structuring the message with system and user roles
    messages = [
        {"role": "user", "content": instruction}
    ]

    # Creating the text-generation pipeline with a deterministic setting
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, temperature=0.9)
    output = pipe(messages)

    # Return the generated response containing the structured JSON analysis
    return output[0]['generated_text']


3. Prompt Testing

3.2.1 Levels of Solidarity run LLM

In [ ]:
import tqdm
subset = df_sentences['sentence'].sample(5, random_state=42).reset_index(drop=True)
# Initialize an empty list for existing labels (for theme extraction)
existing_labels = []

# Process the subset of 20 sentences and store results in a list
solidarity_collect = []
for sentence in subset:
    solidarity = code_text_with_theme(sentence,  model = model, tokenizer = tokenizer)
    solidarity_collect.append(solidarity)


# Create a results dataframe and display it
#results_df = pd.DataFrame(solidarity)
print(solidarity[2]["content"])

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

NameError: name 'results' is not defined

In [ ]:
print(solidarity_collect[2])

[{'role': 'system', 'content': 'You are an expert researcher in political science. Analyze the following text and determine which of the following solidarity categories best describe it: National Solidarity, European-Level Solidarity, Outside-Europe Solidarity. If the text aligns with more than one category, return a comma-separated list of the applicable categories. Return only the category labels with no additional commentary.Write Neutral, if the sentence cannot be linked to solidarity.'}, {'role': 'user', 'content': 'The Council also welcomes the establishment of a study group to identify, in the short term, concrete measures to make more efficient use of existing policies and instruments, in particular in the area of \u200b\u200bcooperation with countries of origin and transit, the activities of the European Agency for the Management of Operational Cooperation at the External Borders of the Member States of the Union (FRONTEX) and the fight against human trafficking and illegal en

3.2.2 Levels of Solidarity Disaggregate Output

In [ ]:
responses = []
for conv in solidarity_collect:
    for message in conv:
        if message.get("role") == "assistant":
            responses.append(message.get("content").strip())
print(responses)
sentence = []
for conv in solidarity_collect:
    for message in conv:
        if message.get("role") == "user":
            sentence.append(message.get("content").strip())
print(sentence)

['Neutral', 'European-Level Solidarity, National Solidarity', 'Outside-Europe Solidarity', 'Neutral', 'Neutral']
['Thank you very much for this visit, Paolo, and welcome to Spain.', 'More recently we have also made important contributions to the design of the Banking Union and to new integration objectives, such as the Common Energy Market, the digital economy, services or the Economic, Fiscal and Political Union projects.', 'The Council also welcomes the establishment of a study group to identify, in the short term, concrete measures to make more efficient use of existing policies and instruments, in particular in the area of \u200b\u200bcooperation with countries of origin and transit, the activities of the European Agency for the Management of Operational Cooperation at the External Borders of the Member States of the Union (FRONTEX) and the fight against human trafficking and illegal entry of migrants.', 'And I propose that we proclaim together the nouns progress, development, coex

3.3.1 Solidarity Speech Acts

In [ ]:
import tqdm
from transformers import pipeline
import pandas as pd
sentences = [
    "In times of crisis, solidarity is our strongest weapon.",
    "True democracy thrives when we stand together as one.",
    "National unity begins with mutual respect and shared purpose.",
    "Let us build a society rooted in compassion and collective strength.",
    "Solidarity is not just a word — it's a commitment to one another.",
    "We rise by lifting others, hand in hand, across all communities.",
    "A just world requires that no one is left behind.",
    "Only through unity can we overcome the challenges of our time.",
    "Our strength lies not in division, but in shared struggle and hope.",
    "Solidarity means fighting for equality, justice, and human dignity for all."
]

# Create DataFrame
df_sentences = pd.DataFrame(sentences, columns=["sentence"])
subset = df_sentences['sentence'].sample(1, random_state=99).reset_index(drop=True)
# Initialize an empty list for existing labels (for theme extraction)
existing_labels = []

# Process the subset of 20 sentences and store results in a list
solidarity_collect = []
for sentence in subset:
    FastLanguageModel.for_inference(model)
    solidarity = code_solidarity_speech_acts(sentence, model, tokenizer)
    solidarity_collect.append(solidarity)


# Create a results dataframe and display it
#results_df = pd.DataFrame(results)
print(solidarity[2]["content"])

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa

IndexError: list index out of range

In [ ]:
print(solidarity_collect)

[[{'role': 'user', 'content': 'You are an expert in political science and computational discourse analysis. Your task is to analyze the following political text to identify and code all instances of solidarity-related speech acts. A valid solidarity-related speech act must meet these criteria: 1. Transitive Structure: The utterance must contain a transitive verb structure linking a subject (the provider, typically the speaker) to a direct object (the recipient). 2. Positive Orientation: The subject expresses a positive orientation (they offer some form of assistance or help, even if verbal) toward the recipient, who is understood to be in a comparatively difficult situation. 3. Illocutionary Type: The speech act must clearly fall into one of the following categories:    - Solidarity Expressives: Utterances that communicate emotional support or compassion without calling for specific action.    - Solidarity Directives: Utterances that demand, request, or call on others to act in support

In [ ]:
import json
results = []
for conv in solidarity_collect:
    original_sentence = None
    for message in conv:
        # Extract the original sentence from the user message for reference
        if message.get("role") == "user":
            original_sentence = message.get("content").strip()
        # Process the assistant's response which should contain JSON data
        elif message.get("role") == "assistant":
            assistant_text = message.get("content").strip()
            try:
                # Parse the JSON array output from the assistant
                speech_acts = json.loads(assistant_text)
                # For each identified speech act, add an entry to the results
                for act in speech_acts:
                    # Optionally include the original sentence for reference
                    act["OriginalSentence"] = original_sentence
                    results.append(act)
            except json.JSONDecodeError:
                print("Error: Could not decode JSON for assistant message:\n", assistant_text)

# Create a DataFrame from the list of dictionaries
df_results = pd.DataFrame(results)

# Display the resulting DataFrame with separate columns for each part of the response
print(df_results)

Error: Could not decode JSON for assistant message:
 You're making a clever transition from the language of solidarity to the language of solidarity.

To continue the analysis:

It seems that the original text was trying to convey the idea that solidarity is about unity and mutual support, rather than division or separation. The correct interpretation of solidarity-related speech acts involves identifying the types of speech acts, such as expressive, directive, or commissive, and understanding the context in which they are used.

Let's break down the key concepts:

*   Speech act: A speech act is a single utterance or message that conveys a particular meaning or intent.
*   Speech act type: Speech act types include expressive, directive, or commissive, which refer to the speaker's intent or the action's effect.
*   Speech act context: The context in which a speech act is used can include the situation, the audience, and the speaker's relationship with the audience.

By understanding th

In [ ]:
# Save DataFrame to CSV file in the current Colab working directory
df_results.to_csv("df_results.csv", index=False)

# If you want to download the file to your local machine, use:
from google.colab import files
files.download("df_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>